<a href="https://colab.research.google.com/github/Lahiru-mta/Rice-Classification-Sri-Lanka/blob/main/version%2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Get authorize access by Earth Engine
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yhK3slRTL4GLYIlzi5BVmvS4aaLwjI_tKZMSspno2z4&tc=WEjHIEtyoVDnua0b7-bU8rRr2lQqJFEafXdOtNmeGPY&cc=LfUngtutfEYmmMZmlFQX8mThr7zG4o5dldfkkPVpRf4

The authorization workflow will generate a code, which you should paste in the box below.


In [ ]:
#@title Connect to Google Drive
#@markdown (To save outputs in the drive)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Import dependencies
!pip install geemap --quiet
!pip install pydub --quiet

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
from IPython.display import clear_output 
from IPython.utils import io
import ipywidgets as widgets
%matplotlib inline
from datetime import datetime, timedelta
import gdown
import geemap
import time

from tqdm.notebook import tqdm

import shutil
import os
import zipfile
from pydub import AudioSegment
from pydub.utils import make_chunks

#for raster manupulation
!pip install rasterio --quiet
import rasterio
from rasterio.merge import merge
import glob
from rasterio.plot import show

#for filtering
!git clone https://github.com/adugnag/gee_s1_ard.git  --quiet
!pip install geemap --quiet

from importlib.machinery import SourceFileLoader

bnc = SourceFileLoader("border_noise_correction", "/content/gee_s1_ard/python-api/border_noise_correction.py").load_module()
sf = SourceFileLoader("speckle_filter", "/content/gee_s1_ard/python-api/speckle_filter.py").load_module()
trf = SourceFileLoader("terrain_flattening", "/content/gee_s1_ard/python-api/terrain_flattening.py").load_module()
wp = SourceFileLoader("wrapper", "/content/gee_s1_ard/python-api/wrapper.py").load_module()
helper = SourceFileLoader("helper", "/content/gee_s1_ard/python-api/helper.py").load_module()

clear_output() 

In [ ]:
#@title Functions

def date_from_img_name (img_1_name):
  name = img_1_name
  date_str = name[4:8] + '-' + name[8:10] + '-' + name[10:]
  date = datetime.fromisoformat(date_str).date()
  return date

def tometers(degrees):
  return(degrees * 111139)

def todegrees(meters):
  return(meters / 111139)

def fn_no_points(p_p_sqr_km, aoi):
  return (int(aoi.area(5).getInfo() * p_p_sqr_km / 1000000))

def find(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

def rectangle(lon_1, lon_2, lat_1, lat_2):
    AOI_coordinates =  [[lon_1, lat_2],
                        [lon_1, lat_1],
                        [lon_2, lat_1],
                        [lon_2, lat_2],
                        [lon_1, lat_2]]

    aoi = ee.Geometry.Polygon(AOI_coordinates)
    return aoi

def sentinel_1_dates (start_date,end_date,aoi):
    img_col = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filterBounds(aoi)
              .filterDate(start_date,end_date)
              .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
              .sort('date'))
    
    # Getting the acquisition dates from the images as text

    time_stamp_list = (img_col.aggregate_array('date')
                    .getInfo())
    # print(time_stamp_list)


    #filter image collections with fewer images

    # convert text dates into datetime
    from pyasn1_modules.rfc2459 import Name
    myset = list(set(time_stamp_list))
    myset.sort()
    # print(myset)

    # count the repititions of the dates
    element_counts = []
    for element in myset:
      element_count = time_stamp_list.count(element)
      element_counts.append(element_count)
      # take the maximun number of repition as the number of images to cover the area
      no_imgs = (max(set(element_counts), key = element_counts.count))


    # remove the dates with less number of images. If two consecutive dates have less images, this wont work.
    # I got no issues about that for sentine 1 data.
    for element in myset:
      element_count = time_stamp_list.count(element)
      if (element_count < no_imgs):
        myset.remove(element)
    # print(myset)
    return myset


def s_img(start_d,end_d):
  #Parameters
  parameter = {  'START_DATE': start_d,
              'STOP_DATE': end_d,        
              'POLARIZATION': band,
              'ORBIT' : 'BOTH',
              'PLATFORM_NUMBER' : None,
              'ORBIT_NUM': None,
              'ROI': c_aoi,
              'APPLY_BORDER_NOISE_CORRECTION': False,
              'APPLY_SPECKLE_FILTERING': apply_filtering,
              'SPECKLE_FILTER_FRAMEWORK':'MONO',
              #  ['BOXCAR', 'LEE', 'GAMMA MAP','REFINED LEE', 'LEE SIGMA']
              'SPECKLE_FILTER': filter,
              'SPECKLE_FILTER_KERNEL_SIZE': Kernal_size,
              'SPECKLE_FILTER_NR_OF_IMAGES':scale,
              'APPLY_TERRAIN_FLATTENING': True,
              'DEM': ee.Image('USGS/SRTMGL1_003'),
              'TERRAIN_FLATTENING_MODEL': 'VOLUME',
              'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
              'FORMAT': 'DB',
              'CLIP_TO_ROI': True,
              'SAVE_ASSET': True,
              'ASSET_ID': "users/mraj"
              }

  #processed s1 collection
  with io.capture_output() as captured:
    s2_processed = wp.s1_preproc(parameter)
  # clear_output()
  return s2_processed

In [ ]:
#@title Import the shape file of the paddy area
#@markdown paste the Google Drive Shared link of the Zip below ex: " https://drive.google.com/file/d/1QKimP8DQl7cgHJ8SpYitwTOwvPGdu-lt/view?usp=sharing "
ZIP_link = 'https://drive.google.com/file/d/1IXs08jU8TJFypvyfINYh5UNO3cJDxsHR/view?usp=sharing'#@param{type : "string"}
# 'https://drive.google.com/uc?id=1P3EFH5v8pOkJOUbDEmB_EpmqIbrHUZHk'
url = 'https://drive.google.com/uc?id=' + ZIP_link[32:65]
File_name = 'field_data' #param{type : "string"}
#@markdown *  ***please make sure the shape file is in WGS84 projection***
out = File_name +'.zip'

d_file_name = gdown.download(url, out, quiet=True)

zip_file = '/content/'+out
path = '/content/'+ File_name
zip_ref = zipfile.ZipFile(zip_file)
try:
    os.mkdir(path)
except:
    print('')
zip_ref.extractall(path)
zip_ref.close()

arr = os.listdir(path)
arr2=[]
for i in range(len(arr)):
  if arr[i].endswith('shp'):
    arr2.append(arr[i])
# print(arr2)

if len(arr2)>1:

  print(d_file_name+' contains '+str(len(arr2))+' shape files\n'
          + 'Select the required shape file below')

  shape_file = widgets.Dropdown(    options=arr2,    value=arr2[0],    description='Shape File:')
  display(shape_file)
else:
  shape_file = arr2[0]

# https://github.com/giswqs/geemap/blob/master/examples/notebooks/10_shapefiles.ipynb
# Refer the following code as well
# https://colab.research.google.com/github/csaybar/EEwPython/blob/master/4_features.ipynb


In [ ]:
#@title Parameters
#@markdown Parameters for the sentinel 1 image collection
Start_Date = '2019-05-01' #@param {type:"date"}
End_Date = '2020-08-01'  #@param {type:"date"}
Threshold = -18.5 #@param {type : 'slider', min:-20, max:0, step:0.1  }
Band = 'VH' #param ['VV','VH']
# bands_list = ['VV','VH']
Scale = 10  #param {type : 'slider', min:0, max:90, step:10  }
Filter = 'LEE SIGMA' #param ['None','BOXCAR', 'LEE', 'GAMMA MAP','REFINED LEE', 'LEE SIGMA']
Kernal_size = 5  #param {type : 'slider', min:3, max:13, step:2  }
padding = 100 #param {type : 'slider', min:0, max:10000, step:100  }
#markdown ###Threshold values
Peaks = -15.5 #param {type : 'slider', min:-30, max:0, step:0.01  }
#markdown Peaks are taken greater than above value
Valleys = -18.5 #param {type : 'slider', min:-30, max:0, step:0.01  }
#markdown Valleys are taken less than above value
Max_tile_size_km = 10 #param {type : 'slider', min:0, max:50, step:5  }
Show_AOI_on_map = False #@param {type:"boolean"}

start_date_i = Start_Date
end_date_i = End_Date
band = Band
scale = Scale
padding_d = todegrees(padding)
threshold = Threshold
threshold_2 = Peaks
threshold_3 = Valleys
tile_size = todegrees(Max_tile_size_km * 1000)
Show_on_Map = Show_AOI_on_map

if Filter == 'None':
  apply_filtering = False
  filter = None
else: 
  filter = Filter
  apply_filtering = True


print('\r','Collecting Data', end = '')

#@title Create an AOI

if len(arr2)>1:
  paddy_shp = path +'/'+ shape_file.value
else:
  paddy_shp = path +'/'+ shape_file
paddy = geemap.shp_to_ee(paddy_shp)

# paddy = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_co", "equals", "CE")

print('\r','Collecting Data.', end = '')

bound_coordinates = paddy.geometry().bounds().coordinates().getInfo()[0]
Upper_left = [a - padding_d for a in bound_coordinates[0]]
Bottom_right = [b + padding_d for b in bound_coordinates[2]]

lon_1 = Upper_left[0]
lon_2 = Bottom_right[0]
lat_1 = Upper_left[1]
lat_2 = Bottom_right[1]

AOI_coordinates =  [[lon_1, lat_2],
                    [lon_1, lat_1],
                    [lon_2, lat_1],
                    [lon_2, lat_2],
                    [lon_1, lat_2]]
      
print('\r','Collecting Data..', end = '')

aoi = ee.Geometry.Polygon(AOI_coordinates)

center = [(lat_1+lat_2)/2,(lon_1+lon_2)/2]
if Show_on_Map == True:
  Map = geemap.Map(center = (center[0], center[1]), zoom=12)
  Map.addLayer(aoi,{},'aoi')
  Map.addLayer(paddy, {}, 'paddy')
  display(Map)

print('\r','Collecting Data...', end = '')


In [ ]:
#@title Process Data

results = []
result_imgs =[]


c_aoi = aoi
c_paddy = paddy

prefix = 'Geometry '

start_date = start_date_i
end_date = end_date_i

  # new img collection
myset = sentinel_1_dates(start_date,end_date,c_aoi)

#create the list of image names
imgs = []
lables_x = []
dates = []

with tqdm(total=len(myset), desc=(prefix +'Collecting Images') , bar_format="{l_bar}{bar} [ time left: {remaining} ]", leave=False) as pbar2:
  for i in range(0,len(myset)) :

    start_Date = myset[i][:4] + '-' + myset[i][4:6] + '-' + myset[i][6:]

    if i<len(myset)-1:
      end_Date = myset[i+1][:4] + '-' + myset[i+1][4:6] + '-' + myset[i+1][6:]
    else: end_Date = end_date

    globals()['img_%s' % myset[i]] = s_img(start_Date,end_Date).mean().select([band])


    imgs.append('img_%s' % myset[i])
    lables_x.append(myset[i][:4] + '-' + myset[i][4:6] + '-' + myset[i][6:])
    dates.append(datetime.fromisoformat(lables_x[i]))
    pbar2.update(1)

# with tqdm(total=len(imgs)*2+len(dates)+1, desc=(prefix +'Process') , bar_format="{l_bar}{bar} [ time left: {remaining} ]", leave=False) as pbar3:
img_set_start_date = myset[0][:4] + '-' + myset[0][4:6] + '-' + myset[0][6:]

#@title Pre-Preprocess

#find min images
a = 0
min_imgs =[]
for a in range(len(imgs)):
  c_img = globals()[imgs[a]]
  b = 0
  temp_min_imgs = []
  for b in range(len(imgs)):
    if a != b:
      c2_img = globals()[imgs[b]]
      globals()['min_img_%s' %b] = c_img.lte(c2_img)
      temp_min_imgs.append('min_img_%s' %b)
  b = b + 1
  c = 0
  for c in range(len(temp_min_imgs)):
    if c == 0:
      min_img = globals()[temp_min_imgs[c]]
    else:
      min_img = min_img.multiply(globals()[temp_min_imgs[c]])
  globals()['min_img_%s' %a] = min_img

  min_imgs.append('min_img_%s' %a)
  a = a + 1

tshd_imgs = []
for img in imgs:
  c_img = globals()[img]
  globals()['tshd_%s' %img] = c_img.lte(threshold)
  tshd_imgs.append('tshd_%s' %img)

d = 0
for d in range(len(tshd_imgs)-4):
  c_img = globals()[tshd_imgs[d]]
  for e in range(d,d+4):
    c_min_img = globals()['min_img_%s' %e] 
    if e == 0:
      f_img = c_img
    else:
      f_img = f_img.add(c_img)
    f_img = f_img.gt(0)

  f_img = f_img.multiply(c_min_img)  

  globals()[tshd_imgs[d]] = f_img

v = 0
sowns = []
sown_dates = []

for j in range(len(tshd_imgs)-4):
  c_img = globals()[tshd_imgs[j]]
  sow_date = dates[j]
  globals()['sown_%s' % str(v)] = c_img
  sowns.append('sown_%s' % str(v))
  v = v + 1
  # pbar3.update(1)
            
dates_str = [str(date.date()) for date in dates]

day_numbers = []     
x = 0
for date in dates_str:
  if x < len(tshd_imgs)-4:
    day_number = datetime.fromisoformat(dates_str[x]).timetuple().tm_yday
    day_numbers.append(day_number)
    if x == 0:
      result = globals()['sown_%s' % str(x)].multiply(day_number)
    else:
      result = result.add((globals()['sown_%s' % str(x)]).multiply((result.gt(0)).eq(0)).multiply(day_number))
    x = x + 1
  # pbar3.update(1)

result = result.add(0).clip(c_paddy)
mask1 = result.lte(0)

kernel1 = ee.Kernel.circle( 0.05 , "pixels", True, 1)
kernel2 = ee.Kernel.rectangle(1, 1, "pixels", True, 1)

result_d = result.focalMax(kernel = kernel1, iterations = 1)
result_d2 = result_d.multiply(mask1)
result_d3 = result.add(result_d2)
result_d3 = result_d3.mask(result_d)
result_e = result_d3.focalMode(kernel = kernel2, iterations = 1)
result_e = result_e.mask(result_e)

result = result_e

result = result.clip(c_paddy)

# pbar3.update(1)
  
# print(min(day_numbers))
# print(max(day_numbers))
print(prefix +'Done', end = '')

Geometry Collecting Images:   0%|           [ time left: ? ]

Geometry Done

In [ ]:
#@title View Result
Map = geemap.Map(center = (center[0], center[1]), zoom=16)
# Map.addLayer(aoi,{},'aoi')
# Map.addLayer(paddy, {}, 'paddy')
Map.addLayer(result.updateMask(result),{ 'min': 200, 'max': 360, 'palette' : ['Red','Blue'] }, 'rslt')
display(Map)

Map(center=[7.873319638445205, 80.7662086530342], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
#@title Export


n_cols = int((lon_2 - lon_1) / tile_size) + 1
n_rows = int((lat_2 - lat_1) / tile_size) + 1
n_tiles = n_cols * n_rows
n_tiles
e_0 = lon_1
e = [e_0]
for aa in range (n_cols):
  globals()['e_%s' % str(aa+1)] = e_0 + (lon_2 - lon_1)/n_cols * (aa+1)
  e.append(globals()['e_%s' % str(aa+1)])
# e
n_0 = lat_1
n = [n_0]
for ab in range (n_rows):
  globals()['e_%s' % str(ab+1)] = n_0 + (lat_2 - lat_1)/n_rows * (ab+1)
  n.append(globals()['e_%s' % str(ab+1)])
# n
ac = 0
aois = []
for ae in range (len(e)-1):
  for an in range (len(n)-1):
      globals()['aoi_%s' % ac] = rectangle(e[ae],e[ae+1],n[an],n[an+1])
      aois.append('aoi_' + str(ac))
      ac = ac +1

print ('\n',len(aois), 'tiles created')

#@title Export Data 
Export_to = '/content/drive/MyDrive/test_5_parak' #@param{type : "string"}
folder = Export_to
try:
    os.mkdir(folder)
except:
    print('path not created')
for roi in aois:
  c_aoi = globals()[roi]
  path = folder+'/'+start_date+'_'+roi+'.tif'
  with io.capture_output() as captured:
  # print(path)
    geemap.ee_export_image(result, filename=path, scale=scale, region=c_aoi, file_per_band=False)


print ('%s TIFF files were saved at %s/' %(start_date,folder))

path2 = folder+'/mosaic_%s.tif' %start_date
search_criteria = "*aoi*.tif"
q = os.path.join(folder, search_criteria)
dem_fps = glob.glob(q)
src_files_to_mosaic = []
for fp in dem_fps:
  src = rasterio.open(fp)
  src_files_to_mosaic.append(src)
mosaic, out_trans = merge(src_files_to_mosaic)

out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
              "height": mosaic.shape[1],
              "width": mosaic.shape[2],
              "transform": out_trans,
              }
              )

with rasterio.open(path2, "w", **out_meta) as dest:
  dest.write(mosaic)

print ('\r','Result was saved at %s ' %path2 , end = '')


 1100 tiles created
path not created
